# Link prediction example for PEG (cora)

In [1]:
import os.path as osp

import torch
import torch.nn.functional as F
from sklearn.metrics import roc_auc_score
from torch_geometric.utils import to_networkx
from torch_geometric.utils import negative_sampling
from torch_geometric.datasets import Planetoid
import torch_geometric.transforms as T
from peg_conv import PEGConv
from torch_geometric.utils import train_test_split_edges
from torch import nn

# Download dataset and load data

In [16]:
device = f'cuda:{1}' if torch.cuda.is_available() else 'cpu'

In [3]:
dataset = 'Cora'
path = osp.join('.', 'data', dataset)
dataset = Planetoid(path, dataset, transform=T.NormalizeFeatures())
data = dataset[0]
print(dataset.data)

Data(x=[2708, 1433], edge_index=[2, 10556], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708])


In [4]:
data.train_mask = data.val_mask = data.test_mask = data.y = None
data = train_test_split_edges(data)
print(data)

/scratch1/wang5386/py2021/lib/python3.7/site-packages/torch_geometric/deprecation.py:13: UserWarning: 'train_test_split_edges' is deprecated, use 'transforms.RandomLinkSplit' instead
  warnings.warn(out)


Data(x=[2708, 1433], val_pos_edge_index=[2, 263], test_pos_edge_index=[2, 527], train_pos_edge_index=[2, 8976], train_neg_adj_mask=[2708, 2708], val_neg_edge_index=[2, 263], test_neg_edge_index=[2, 527])


# Preprocessing: calculate positional encoding (Deepwalk)

In [5]:
#Build train matrix for PE preparation
import copy
train_graph = copy.deepcopy(dataset[0])
train_graph.edge_index = data.train_pos_edge_index
G = to_networkx(train_graph)

In [6]:
import networkx as nx
import pandas as pd
import numpy as np
import random
from tqdm import tqdm
import itertools
import math
from joblib import Parallel, delayed
from tqdm import trange

In [7]:
def partition_num(num, workers):
    if num % workers == 0:
        return [num//workers]*workers
    else:
        return [num//workers]*workers + [num % workers]

In [8]:
#modified from https://github.com/shenweichen/GraphEmbedding
class RandomWalker:
    def __init__(self, G, p=1, q=1, use_rejection_sampling=0):
        """
        :param G:
        :param p: Return parameter,controls the likelihood of immediately revisiting a node in the walk.
        :param q: In-out parameter,allows the search to differentiate between “inward” and “outward” nodes
        :param use_rejection_sampling: Whether to use the rejection sampling strategy in node2vec.
        """
        self.G = G
        self.p = p
        self.q = q
        self.use_rejection_sampling = use_rejection_sampling
    
    def deepwalk_walk(self, walk_length, start_node):

        walk = [start_node]

        while len(walk) < walk_length:
            cur = walk[-1]
            cur_nbrs = list(self.G.neighbors(cur))
            if len(cur_nbrs) > 0:
                walk.append(random.choice(cur_nbrs))
            else:
                break
        return walk

    def simulate_walks(self, num_walks, walk_length, workers=1, verbose=0):

        G = self.G

        nodes = list(G.nodes())

        results = Parallel(n_jobs=workers, verbose=verbose, )(
            delayed(self._simulate_walks)(nodes, num, walk_length) for num in
            partition_num(num_walks, workers))

        walks = list(itertools.chain(*results))

        return walks
    
    def _simulate_walks(self, nodes, num_walks, walk_length,):
        walks = []
        for _ in range(num_walks):
            random.shuffle(nodes)
            for v in nodes:
                if self.p == 1 and self.q == 1:
                    walks.append(self.deepwalk_walk(
                        walk_length=walk_length, start_node=v))
                else:
                    return ("only work for DeepWalk")
        return walks

In [9]:
from gensim.models import Word2Vec
import pandas as pd

class DeepWalk:
    def __init__(self, graph, walk_length = 80, num_walks = 10, workers=1):

        self.graph = graph
        self.w2v_model = None
        self._embeddings = {}

        self.walker = RandomWalker(
            graph, p=1, q=1, )
        self.sentences = self.walker.simulate_walks(
            num_walks=num_walks, walk_length=walk_length, workers=workers, verbose=1)

    def train(self, embed_size=128, window_size=5, workers=3, iter=3, **kwargs):

        kwargs["sentences"] = self.sentences
        kwargs["min_count"] = kwargs.get("min_count", 0)
        kwargs["vector_size"] = embed_size
        kwargs["sg"] = 1  # skip gram
        kwargs["hs"] = 1  # deepwalk use Hierarchical Softmax
        kwargs["workers"] = workers
        kwargs["window"] = window_size
        kwargs["epochs"] = iter

        print("Learning embedding vectors...")
        model = Word2Vec(**kwargs)
        print("Learning embedding vectors done!")

        self.w2v_model = model
        return model

    def get_embeddings(self,):
        if self.w2v_model is None:
            print("model not train")
            return {}

        self._embeddings = {}
        for word in self.graph.nodes():
            self._embeddings[word] = self.w2v_model.wv[word]

        return self._embeddings

In [10]:
model_emb = DeepWalk(G,walk_length=80, num_walks=10,workers=1)#init model
model_emb.train(embed_size = 128)# train model
emb = model_emb.get_embeddings()# get embedding vectors
embeddings = []
for i in range(len(emb)):
    embeddings.append(emb[i])
embeddings = np.array(embeddings)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    2.7s finished


Learning embedding vectors...
Learning embedding vectors done!


# Training: Set up model and train

In [11]:
class Net(torch.nn.Module):
    def __init__(self, in_feats_dim, hidden_dim):
        super(Net, self).__init__()
        
        self.in_feats_dim = in_feats_dim
        self.hidden_dim = hidden_dim

        self.conv1 = PEGConv(in_channels = in_feats_dim, out_channels = hidden_dim)
        self.conv2 = PEGConv(in_channels = hidden_dim, out_channels = hidden_dim)
        self.loss_fn = torch.nn.BCEWithLogitsLoss()
        self.fc = nn.Linear(2, 1)

    def forward(self, x, pos, pos_edge_index, neg_edge_index):
        edge_index = torch.cat([pos_edge_index, neg_edge_index], dim=-1)
        x = self.conv1(x, pos, pos_edge_index)
        x = self.conv2(x, pos, pos_edge_index)
        
        nodes_first = x[edge_index[0]]
        nodes_second = x[edge_index[1]]
        pos_first = pos[edge_index[0]]
        pos_second = pos[edge_index[1]]
        
        positional_encoding = ((pos_first - pos_second)**2).sum(dim=-1, keepdim=True)

        pred = (nodes_first * nodes_second).sum(dim=-1)  # dot product 
        out = self.fc(torch.cat([pred.reshape(len(pred), 1),positional_encoding.reshape(len(positional_encoding), 1)], 1))

        return out

    def loss(self, pred, link_label):
        return self.loss_fn(pred, link_label)

In [19]:
x = data.x
pos = torch.tensor(embeddings).to(device)
x = x.cuda(device)

In [13]:
def get_link_labels(pos_edge_index, neg_edge_index):
    # returns a tensor:
    # [1,1,1,1,...,0,0,0,0,0,..] with the number of ones is equel to the lenght of pos_edge_index
    # and the number of zeros is equal to the length of neg_edge_index
    E = pos_edge_index.size(1) + neg_edge_index.size(1)
    link_labels = torch.zeros(E, dtype=torch.float, device=device)
    link_labels[:pos_edge_index.size(1)] = 1.
    return link_labels


def train():
    model.train()

    neg_edge_index = negative_sampling(
        edge_index=data.train_pos_edge_index, #positive edges
        num_nodes=data.num_nodes, # number of nodes
        num_neg_samples=data.train_pos_edge_index.size(1)) # number of neg_sample equal to number of pos_edges

    optimizer.zero_grad()
    
    link_logits = model(x, pos, data.train_pos_edge_index, neg_edge_index) # decode
    link_logits = link_logits.reshape(len(link_logits),)
    link_labels = get_link_labels(data.train_pos_edge_index, neg_edge_index)
    loss = F.binary_cross_entropy_with_logits(link_logits, link_labels)
    loss.backward()
    optimizer.step()
    with torch.no_grad():
        model.fc.weight[0][0].clamp_(1e-5,100)
    return loss


@torch.no_grad()
def test():
    model.eval()
    perfs = []
    for prefix in ["val", "test"]:
        pos_edge_index = data[f'{prefix}_pos_edge_index']
        neg_edge_index = data[f'{prefix}_neg_edge_index']

        link_logits = model(x, pos, pos_edge_index, neg_edge_index) # decode test or val
        
        link_probs = link_logits.sigmoid() # apply sigmoid
        
        link_labels = get_link_labels(pos_edge_index, neg_edge_index) # get link
        
        perfs.append(roc_auc_score(link_labels.cpu(), link_probs.cpu())) #compute roc_auc score
    return perfs


In [28]:
model = Net(in_feats_dim = dataset.num_features, hidden_dim = 128)
data = data.to(device) 
model = model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr= 0.01, weight_decay= 1e-4)

In [29]:
best_val_perf = test_perf = 0
for epoch in range(1, 101):
    train_loss = train()
    val_perf, tmp_test_perf = test()
    if val_perf > best_val_perf:
        best_val_perf = val_perf
        test_perf = tmp_test_perf
    log = 'Epoch: {:03d}, Loss: {:.4f}, Val: {:.4f}, Test: {:.4f}'
    if epoch % 10 == 0:
        print(log.format(epoch, train_loss, best_val_perf, test_perf))

Epoch: 010, Loss: 0.2796, Val: 0.9022, Test: 0.9170
Epoch: 020, Loss: 0.0455, Val: 0.9022, Test: 0.9170
Epoch: 030, Loss: 0.0514, Val: 0.9022, Test: 0.9170
Epoch: 040, Loss: 0.0527, Val: 0.9022, Test: 0.9170
Epoch: 050, Loss: 0.0427, Val: 0.9022, Test: 0.9170
Epoch: 060, Loss: 0.0392, Val: 0.9022, Test: 0.9170
Epoch: 070, Loss: 0.0381, Val: 0.9022, Test: 0.9170
Epoch: 080, Loss: 0.0330, Val: 0.9022, Test: 0.9170
Epoch: 090, Loss: 0.0333, Val: 0.9022, Test: 0.9170
Epoch: 100, Loss: 0.0333, Val: 0.9022, Test: 0.9170
